In [5]:
import boto3
import uproot
import os
import time
import tempfile
from dask.distributed import Client

In [6]:
cl = Client()

In [7]:
s3 = boto3.client(
    's3',
)

In [8]:
s3.download_file(
    "hepaccelerate-hmm-skim-merged",
    "files.txt",
    "files.txt"
)

ClientError: An error occurred (403) when calling the HeadObject operation: Forbidden

In [5]:
fl = [f.strip() for f in open("files.txt").readlines()]

In [43]:
def get_num_events(fn):
    s3 = boto3.client(
        's3',
    )
    
    t0 = time.time()
    tmp = tempfile.mktemp()
    s3.download_file(
        "hepaccelerate-hmm-skim-merged",
        fn[1:],
        tmp
    )
    tf = uproot.open(tmp)
    tt = tf.get("Events")
    nev = len(tt)
    file_size = os.path.getsize(tmp)
    os.remove(tmp)
    t1 = time.time()
    
    ret = {
        "num_events": nev,
        "file_size": file_size,
        "time_delta": t1 - t0
    }
    
    return ret

In [44]:
rets = cl.map(get_num_events, fl[:100])

In [45]:
rets2 = [r.result() for r in rets]

ModuleNotFoundError: No module named 'boto3'